In [1]:
import os
import sys
import datetime as dt
#import missingno as mno 
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests
from statsmodels.tsa.api import VAR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from numpy import log

In [3]:
df= pd.read_csv("soil.csv")
df.set_index(['date'],inplace= True)
df= df.rename(columns= {"segment1(10-30cm)":"moisture"})

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23267 entries, 2022-06-01 00:08:00+00:00 to 2022-10-31 23:58:00+00:00
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Magna_6 Wind Direction                   23267 non-null  float64
 1   Magna_6 Wind Speed (m/s)                 23267 non-null  float64
 2   Magna_6 Meteo Ambient Temperature (C)    23267 non-null  float64
 3   Magna_6 Meteo Air Pressure (hpa)         23267 non-null  float64
 4   Magna_6 Meteo Relative Humidity          23267 non-null  float64
 5   Magna_6 Meteo Dew Point Temperature (C)  23267 non-null  float64
 6   Power Supply (V)                         23267 non-null  float64
 7   Magna_6 Water EC muS/cm                  23267 non-null  float64
 8   Magna_6 Water Level Above Sensor mm      23267 non-null  float64
 9   moisture                                 23267 non-null  float64
 10  segment

In [5]:
df.loc[df.index.duplicated(),: ]

,Magna_6 Wind Direction,Magna_6 Wind Speed (m/s),Magna_6 Meteo Ambient Temperature (C),Magna_6 Meteo Air Pressure (hpa),Magna_6 Meteo Relative Humidity,Magna_6 Meteo Dew Point Temperature (C),Power Supply (V),Magna_6 Water EC muS/cm,Magna_6 Water Level Above Sensor mm,moisture,...,Magna_6 PH,Magna_6 ORP_mV,Magna_6 pH_Sensor_Temperature_C,Magna_6 Precipitation_24hr_mm,Magna_6 Cumulative Precipitation_mm,Magna_6 CTD_Temperature_C,CTD_Pressure_BAR,CTD_Conductivity__mS,Metres above MSL,TOW _MH2O
date,,,,,,,,,,,,,,,,,,,,,


In [ ]:
#%pip install xgboost

Pycaret TS Model
--

In [6]:
def what_is_installed():
    import pycaret
    from pycaret import show_versions
    show_versions()

try:
    what_is_installed()
except:
  #pip install prophet
   # !pip install pycaret-ts-alpha
    what_is_installed()


System:
    python: 3.8.15 (default, Nov 24 2022, 09:04:07)  [Clang 14.0.6 ]
executable: /Users/sage/opt/anaconda3/envs/sage/bin/python
   machine: macOS-10.16-x86_64-i386-64bit

Python dependencies:
          pip: 22.3.1
   setuptools: 65.5.0
      pycaret: 3.0.0
      sklearn: 1.2.0
       sktime: 0.10.1
  statsmodels: 0.13.5
        numpy: 1.21.6
        scipy: 1.7.3
       pandas: 1.4.4
   matplotlib: 3.6.2
       plotly: 5.11.0
       joblib: 1.2.0
        numba: 0.55.2
       mlflow: 2.0.1
     lightgbm: 3.3.3
      xgboost: 1.7.2
     pmdarima: 1.8.5
        tbats: 1.1.2
      prophet: 1.1.1
      tsfresh: 0.20.0


In [7]:
from pycaret.time_series import TSForecastingExperiment
# Change renderer appropriately based on where the notebook is being run ----
# Refer to plotly for available renderers.
global_plot_settings = {"renderer": "jupyterlab",
                        "hoverinfo": "text"}

# init environment

exp = TSForecastingExperiment()
target = 'moisture'
data_for_modeling = df[df.index < '2022-09-21'].reset_index(drop = True)
future_df = df[df.index >= '2022-09-21'].reset_index(drop = True)
future_exog = future_df.drop(columns=target)

exp.setup(data = data_for_modeling,
          target= target,
          seasonal_period=12,
          fh=2000,
          fig_kwargs=global_plot_settings,
          session_id=140)

ImportError: cannot import name '_pprint' from 'sklearn.base' (/Users/sage/opt/anaconda3/envs/sage/lib/python3.8/site-packages/sklearn/base.py)

In [ ]:
# Additional plots provided by PyCaret to help understand what is being done.
exp.plot_model(plot="ts")

In [ ]:
#Model training and selection
print("Model training and selection")
#exclude={'arima','auto_arima'}
#best_model = exp.compare_models(sort ="MAE", exclude=exclude)
best_model= exp.create_model('lr_cds_dt')

In [ ]:
exp.plot_model(best_model,
               plot = 'forecast')

In [ ]:
# Diagnostics plot
exp.plot_model(plot = 'diagnostics')

In [ ]:
#tune model
tuned= exp.tune_model(best_model,fold=5)


In [ ]:
# forecast in unknown future
exp.plot_model(tuned,
               plot = 'forecast',
               data_kwargs = {'fh' : 2000})

In [ ]:
#from sktime.forecasting.base import *
# Decomposition plot
exp.plot_model(tuned, plot="decomp")
exp.plot_model(tuned, plot="decomp_stl")

In [ ]:
# finalize model
final_best =exp.finalize_model(tuned)
print(final_best)

In [ ]:
future_exog.count


In [ ]:
# generate predictions
preds= exp.predict_model(final_best,fh=5892,X=future_exog)
print(preds)

In [ ]:
y_train= data_for_modeling['moisture']
#y_train=exp.get_config("y_train")
print(y_train)

In [ ]:

data=df['moisture'].reset_index(drop=True)
from sktime.utils.plotting import plot_series
plot_series(data,
            y_train,
            preds,
            labels=['All', 'Train', 'Test Predictions'])
plt.show()

In [ ]:
# save the model
exp.save_model(final_best, 'my_best_model')

BORUTA
--

In [ ]:
#Installing Boruta
#!pip install Boruta
# Boruta for feature selection
#!pip install boruta_py
from sklearn.ensemble import RandomForestRegressor
from boruta import  BorutaPy
from sklearn.metrics import accuracy_score 

In [ ]:
X_new = df.drop(columns =['moisture']).reset_index(drop=True)
X_new

In [ ]:
y_new = df['moisture'].reset_index(drop=True)
y_new

In [ ]:
#Initializing the random forest model
model = RandomForestRegressor(n_estimators = 100,
                              max_depth = 5,
                              random_state = 42)
# let's initialize Boruta
feat_selector = BorutaPy(
    verbose=2,
    estimator=model,
    n_estimators='auto',
    max_iter=10  # number of iterations to perform
)

In [ ]:
#Training the boruta, X and Y mus be numpy arrays
import numpy as np
feat_selector.fit(np.array(X_new), np.array(y_new))

In [ ]:
accepted = X_new.columns[feat_selector.support_].to_list()
print('Accepted features:', accepted)

In [ ]:
undecided = X_new.columns[feat_selector.support_weak_].to_list()
print('Undecided features', undecided)

In [ ]:
selected_features = accepted + undecided # include undecided for safe measure
df_new=df.copy()
df_boruta = df_new.loc[:,selected_features]
df_boruta['moisture']= df['moisture']
df_boruta

In [ ]:
corr=df_boruta.copy().corr()
corr

In [ ]:
import seaborn as sns
sns.set(rc = {'figure.figsize':(20, 12)})
sns.heatmap(corr, annot=True)
plt.show()

In [ ]:

highly_corr = {}
for columns in corr.columns:
  highly_corr[columns] = list(corr[columns][corr[columns]> 0.99].index)
highly_corr


In [ ]:
upper_tri = corr.where(np.triu(np.ones(corr.shape),k=1).astype(np.bool))
upper_tri.info()

In [ ]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]
to_drop

In [ ]:
df_boruta2 = df_boruta.drop(df_boruta[to_drop], axis=1)
df_boruta2['moisture']= df['moisture']
df_boruta2.info()

In [ ]:
from pycaret.time_series import TSForecastingExperiment

# Change renderer appropriately based on where the notebook is being run ----
# Refer to plotly for available renderers.
global_plot_settings = {"renderer": "jupytelab",
                        "hoverinfo": "text"}
# init environment
from pycaret.regression import *
exp_b = TSForecastingExperiment()
target='moisture'
data_for_boruta = df_boruta2[df_boruta2.index < '2022-09-21'].reset_index(drop = True)
future_boruta = df_boruta2[df_boruta2.index >= '2022-09-21'].reset_index(drop = True)
future_exog_boruta = future_boruta.drop(columns=target)

exp_b.setup(data = data_for_boruta,
          target= target,
          seasonal_period=12,
          fh=2000,
          fig_kwargs=global_plot_settings,
          session_id=143)

In [ ]:
exp_b.models()

In [ ]:
#compare models with boruta data
#best_boruta= exp_b.compare_models(sort ="MAE", exclude={'arima','auto_arima','lightgbm_cds_dt'})# remove the slow models and the ones causing the kernel to crash
best_boruta= exp_b.create_model('omp_cds_dt')

In [ ]:
# Additional plots provided by PyCaret to help understand what is being done.
exp_b.plot_model(plot="train_test_split")

In [ ]:
#tune model
tuned_boruta= exp_b.tune_model(best_model,fold=5)


In [ ]:
# forecast in unknown future
exp_b.plot_model(tuned_boruta,
               plot = 'forecast',
               data_kwargs = {'fh' : 2000})

In [ ]:
# Diagnostics plot
exp_b.plot_model(tuned_boruta, plot = 'diagnostics')

In [ ]:
# finalize model
final_boruta = exp_b.finalize_model(tuned_boruta)
print(final_boruta)

In [ ]:
# generate predictions
preds_boruta= exp_b.predict_model(final_boruta,fh=5892,X=future_exog_boruta)
print(preds_boruta)

In [ ]:
y_train= data_for_boruta['moisture']
data=df_boruta2['moisture'].reset_index(drop=True)
from sktime.utils.plotting import plot_series
plot_series(data,
            y_train,
            preds,
            labels=['All', 'Train', 'Test Predictions'])
plt.grid()

In [ ]:
# save the model
exp_b.save_model(final_boruta, 'my_boruta_model')

In [ ]:
normal=preds.reset_index()
normal

In [ ]:
boruta=preds_boruta.reset_index()
boruta= boruta.rename(columns= {"y_pred":"boruta"})

In [ ]:
future_boruta['moisture']

In [ ]:
predictions= pd.concat([future_boruta['moisture'],normal['y_pred'],boruta['boruta']], axis= 1)

In [ ]:
#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)


In [ ]:
predictions.iplot()

In [ ]:

# create ap
exp.create_api(final_best, 'soil_api')


# create app
exp.create_app(final_best)


# create app
exp.create_app(final_best)

In [ ]:
final_boruta

In [ ]:
# run api
!python app.py


from ipywidgets import widgets
exp.evaluate_model('lr_cds_dt')

In [ ]:
help(plot_model)